<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/Hawaii_Soils_Process_and_Intersect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install.packages('sf')
# install.packages('raster')
# install.packages('exactextractr')
# install.packages("googledrive")
# install.packages("googletoken")
# install.packages("DBI")
# install.packages("RSQLite")
# install.packages("RPostgreSQL")
# install.packages("terra")
# install.packages('randomForest')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("sf"):
“installation of package ‘sf’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message in install.packages("raster"):
“installation of package ‘raster’ had non-zero exit status”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
library(ggplot2)
library(sf)
library(magrittr)
library(data.table)
library(raster)
library(exactextractr)
library(googledrive)
library(DBI) # database
library(terra)
library(randomForest)
library(parallel)

Linking to GEOS 3.11.1, GDAL 3.6.4, PROJ 9.1.1; sf_use_s2() is TRUE

Loading required package: sp

terra 1.7.78


Attaching package: ‘terra’


The following object is masked from ‘package:data.table’:

    shift


The following objects are masked from ‘package:magrittr’:

    extract, inset


randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:ggplot2’:

    margin




In [3]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=9129c80703b4899ebec9d157d0559b1c&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVFsRWQ4ekNlYmQySVgzcmRKUFdUX2FSMkxSeVR6WklyYzVVR0Y0U2ZGVWdnVnJJQ1FVb2ZDNmZiWXZhVnh2ZGdUbkNuQSIsInN0YXRlIjoiOTEyOWM4MDcwM2I0ODk5ZWJlYzlkMTU3ZDA1NTliMWMifQ==


In [4]:
dir.create('/content/predictor_variables/')

Warning message in dir.create("/content/predictor_variables/"):
“'/content/predictor_variables' already exists”


In [5]:
# Hawaii GEDI data
# https://drive.google.com/file/d/1UY3lxML0Y7bat_9-xEZQQHDkYRcODDZy/view?usp=drive_link
drive_download(file = as_id('1UY3lxML0Y7bat_9-xEZQQHDkYRcODDZy'), path =  '/content/predictor_variables/Hawaii_GEDI_4A.gpkg', overwrite = FALSE)
gedi_sf <- st_read('/content/predictor_variables/Hawaii_GEDI_4A.gpkg')

ERROR: [1m[33mError[39m in `drive_download()`:[22m
[1m[22m[33m![39m Local `path` already exists and overwrite is `FALSE`:
[36m•[39m [34m/content/predictor_variables/Hawaii_GEDI_4A.gpkg[39m


In [ ]:
# All islands 250m grid
# https://drive.google.com/file/d/1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz/view?usp=drive_link
drive_download(file = as_id('1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz'), path =  '/content/predictor_variables/HI_250_m_grid.gpkg', overwrite = FALSE)

In [ ]:
folder_loc <- '/content/predictor_variables/TerraClimate/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1bIbwKYmzP2o6VEAWyLIRmeMQbGFDt-vX?usp=drive_link
folder_id <- "1bIbwKYmzP2o6VEAWyLIRmeMQbGFDt-vX"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}


replace files in the Drive folder

In [ ]:
folder_loc <- '/content/predictor_variables/SRTM_mTPI/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1YKJrZXTo-L4UpvJ0iEGo8yIX6uhoQ9AW?usp=drive_link
folder_id <- "1YKJrZXTo-L4UpvJ0iEGo8yIX6uhoQ9AW"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

In [ ]:
folder_loc <- '/content/predictor_variables/Elevation/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1XmH9T9xVINZZfPFi39Wus1l7c0SKS4Am?usp=drive_link
folder_id <- "1XmH9T9xVINZZfPFi39Wus1l7c0SKS4Am"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

In [ ]:
folder_loc <- '/content/predictor_variables/LandForms/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1PkUR4GK50wC6Kgrvyy-d6067ypdX1dkx?usp=drive_link
folder_id <- "1PkUR4GK50wC6Kgrvyy-d6067ypdX1dkx"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

In [ ]:
folder_loc <- '/content/predictor_variables/LandSat_composite/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/1NzvurSghdEJdX0UsfdyuOcOyh6n3jUDo?usp=drive_link
folder_id <- "1NzvurSghdEJdX0UsfdyuOcOyh6n3jUDo"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

In [ ]:
folder_loc <- '/content/predictor_variables/DynamicWorld_landcover/'
dir.create(folder_loc)

# https://drive.google.com/drive/folders/152pjTArpwlDVGjXFoSU9bmo1BDSKdCqq?usp=drive_link
folder_id <- "152pjTArpwlDVGjXFoSU9bmo1BDSKdCqq"
folder_files <- drive_ls(as_id(folder_id))

for (i in 1:nrow(folder_files)) {
  file_id <- folder_files$id[i]
  file_name <- folder_files$name[i]
  drive_download(as_id(file_id), path = paste0(folder_loc,file_name), overwrite = TRUE)
}

In [ ]:
  # https://drive.google.com/file/d/1ryTzWz0t9mgI5ynPvMzM21D135dLJvWv/view?usp=drive_link
  # Updated: https://drive.google.com/file/d/17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C/view?usp=sharing
# drive_download(as_id('17c3DJcujQ_0MHirG2z2lSU88A0lvjo1C'), path = paste0('/content/fia.csv'), overwrite = TRUE)
# fia_dt <-  st_read('/content/fia.csv') %>% st_as_sf(coords = c("LON","LAT"), crs = 4326) %>% data.table

In [6]:
dir_list <- list.dirs('/content/predictor_variables')[-1]

names(dir_list) <- list.dirs('/content/predictor_variables',full.names = FALSE)[-1]

vrt is making a virtual raster tile
option to fix the issue: make a vrt individually for each island then stack them after
see if terraclimate is the issue (for Kauai), getting hidden by Oahu data

In [7]:
# List of your TIFF files
tif_files <- list.files("/content/predictor_variables/TerraClimate", pattern = "\\.tif$", full.names = TRUE)

# Read the TIFF files into a list of raster objects
raster_list <- lapply(tif_files, rast)

mosaic_raster <- do.call(mosaic, c(raster_list, fun = max))

writeRaster(mosaic_raster, "/content/predictor_variables/TerraClimate.tif", overwrite = TRUE)

In [8]:
names(dir_list) %>% lapply(function(x){
# print(x)
 y <- dir_list[[x]]
 vrt(y %>% lapply(function(x){
  file_list <- list.files(y, pattern='*.tif',full.names = T)
  file_sizes <- lapply(file_list,file.size) %>% unlist %>% order
  file_list[file_sizes] %>% return
  }) %>% unlist,filename=paste0(y,'/',x,'.vrt'),overwrite=T)
#  vrt_rast <- paste0(y,'/',x,'.vrt') %>% rast
#  writeRaster(vrt_rast, paste0('/content/predictor_mosaic/',x,'.tif'),overwrite=T)
#  print(paste('Done',x))
})

In [ ]:
list.files('/content/predictor_variables',pattern = '*.vrt',full.names = T,recursive = T)

In [9]:
raster_list <- list.files('/content/predictor_variables',pattern = '*.vrt',full.names = T,recursive = T) %>% lapply(stack) %>% lapply(rast)

raster_list

[[1]]
class       : SpatRaster 
dimensions  : 371833, 2204931, 10  (nrow, ncol, nlyr)
resolution  : 9.031842e-05, 9.031842e-05  (x, y)
extent      : -178.4436, 20.70229, 18.8654, 52.44877  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      : DynamicWorld_landcover.vrt 
names       : Dynam~ver_1, Dynam~ver_2, Dynam~ver_3, Dynam~ver_4, Dynam~ver_5, Dynam~ver_6, ... 

[[2]]
class       : SpatRaster 
dimensions  : 35816, 87898, 1  (nrow, ncol, nlyr)
resolution  : 0.0002694946, 0.0002694946  (x, y)
extent      : -178.4436, -154.7556, 18.8652, 28.51742  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      : Elevation.vrt 
name        : Elevation 

[[3]]
class       : SpatRaster 
dimensions  : 80530, 270460, 1  (nrow, ncol, nlyr)
resolution  : 0.0002694946, 0.0002694946  (x, y)
extent      : -178.4437, -105.5562, 18.86543, 40.56783  (xmin, xmax, ymin, ymax)
coord. ref. : +proj=longlat +datum=WGS84 +no_defs 
source      : 

In [10]:
raster_list[[6]] <- rast('/content/predictor_variables/TerraClimate.tif')

In [11]:
gedi_dt <- st_read('/content/predictor_variables/Hawaii_GEDI_4A.gpkg') %>% data.table()

Reading layer `Hawaii_GEDI_2' from data source 
  `/content/predictor_variables/Hawaii_GEDI_4A.gpkg' using driver `GPKG'
Integer64 values larger than 9.0072e+15 lost significance after conversion to double;
use argument int64_as_string = TRUE to import them lossless, as character
Simple feature collection with 4839466 features and 28 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -160.6006 ymin: 18.86556 xmax: -154.7559 ymax: 22.28996
Geodetic CRS:  WGS 84


In [12]:
#https://drive.google.com/file/d/1DcBOzWHwqJovd9hhIqLUjRlc2kpqEGKz/view?usp=drive_link
grid_dt <- st_read('/content/predictor_variables/HI_250_m_grid.gpkg') %>% data.table()

Reading layer `250_summary_grid_dt' from data source 
  `/content/predictor_variables/HI_250_m_grid.gpkg' using driver `GPKG'
Simple feature collection with 359163 features and 35 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -159.7877 ymin: 18.91217 xmax: -154.807 ymax: 22.23237
Geodetic CRS:  WGS 84


In [13]:
example_poly <- grid_dt %>% st_as_sf %>% st_transform(4135) %>% st_buffer(250) %>% st_transform(4326)

In [14]:
  cover_dt <- data.table(exact_extract(raster_list[[1]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(cover_dt) <- c("water", "trees", "grass", "flooded_vegetation", "crops", "shrub_and_scrub", "built", "bare", "snow_and_ice",'max')

  elevation_dt <- data.table(exact_extract(raster_list[[2]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(elevation_dt) <- c("elevation")

  landform_dt <- data.table(exact_extract(raster_list[[3]],example_poly %>% st_as_sf,'mode',progress = TRUE) %>% data.table)
  names(landform_dt) <- c("landform")

  ls_dt <- data.table(exact_extract(raster_list[[4]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(ls_dt) <- c("ultra_blue", "blue", "green", "red", "infrared", "sw1", "sw2")

  tpi_dt <- data.table(exact_extract(raster_list[[5]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(tpi_dt) <- c("SRTM_mTPI")

  climate_dt <- data.table(exact_extract(raster_list[[6]],example_poly %>% st_as_sf,'mean',progress = TRUE) %>% data.table)
  names(climate_dt) <- c("aet", "def", "pdsi", "pet", "pr", "ro", "soil", "srad", "swe", "tmmn", "tmmx", "vap", "vpd", "vs")


Cannot preload entire working area of 20320776000 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%


Cannot preload entire working area of 228265338 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%


Cannot preload entire working area of 228246837 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%


Cannot preload entire working area of 1597727859 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.



  |======================================================================| 100%
  |======================================================================| 100%
  |======================================================================| 100%


In [15]:
gedi_sf %>% data.table %>% names

ERROR: Error in eval(expr, envir, enclos): object 'gedi_sf' not found


In [ ]:
# st_crs(gedi_sf)
# grid_int_dt <- fia_dt[,c('PLOT','Stock_SOC_Mg_Ha','geometry')] %>% st_as_sf() %>% st_transform(4135) %>% st_buffer(10000) %>% st_transform(4326) %>% st_intersects(grid_dt %>% st_as_sf %>% st_transform(4326))

In [ ]:
# gedi_sf$l4_quality_flag %>% table
# gedi_dt <-gedi_sf %>% data.table
gedi_dt <- gedi_dt[l4_quality_flag == 1]

In [ ]:
gedi_int_dt <- gedi_dt$geom %>% st_intersects(example_poly,.)

In [ ]:
example_poly[1,] %>% st_intersects(gedi_sf)

In [ ]:
gedi_summary_dt <- gedi_int_dt %>% lapply(function(x){
  sub_gedi_dt <- gedi_dt[x,]
  sub_gedi_dt[,list(agbd_m = mean(agbd,na.rm=T),agbd_sd = sd(agbd,na.rm=T),agbd_n = length(agbd))]
}) %>% rbindlist()

In [ ]:
# gedi_summary_dt %>% nrow

In [ ]:
rf_dt <- data.table(example_poly,cover_dt,elevation_dt,landform_dt,tpi_dt,climate_dt,gedi_summary_dt)


In [ ]:
# rf_dt$geom <- grid_dt[grid_int_unique]$geom


In [ ]:
rf_dt %>% st_as_sf %>% st_write('250_summary_grid_dt.gpkg',delete_dsn = T)

In [ ]:
rf_dt %>% names

In [ ]:
rf_dt %>%
  ggplot(aes(x=srad,y=agbd_m)) +
  # xlim(0,500)+

  geom_bin2d() +
    geom_smooth(method='lm')

In [ ]:
# https://drive.google.com/drive/folders/1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM?usp=drive_link
drive_upload("/content/250_summary_grid_dt.gpkg", path = as_id("1uVZCzCb5gRE8wfZckspRuKz1TQd5LpeM"))